In [1]:
using Plots
pyplot()

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1192


ArgumentError: ArgumentError: Package PyPlot not found in current path:
- Run `import Pkg; Pkg.add("PyPlot")` to install the PyPlot package.


In [7]:
pyplot()

┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1192


Plots.PyPlotBackend()

In [11]:
using Primes

┌ Info: Precompiling Primes [27ebfcd6-29c5-5fa9-bf4b-fb8fc14df3ae]
└ @ Base loading.jl:1192


In [1]:
using Distributed
# Script for investigating amplitude dependence of potential
@everywhere using Distributions
@everywhere using Test
@everywhere using StatsBase
@everywhere using BenchmarkTools
@everywhere using LinearAlgebra
@everywhere using LaTeXStrings
@everywhere using LinearAlgebra
using Primes
using MCMCDiagnostics
using SharedArrays
using Plots
pyplot()

┌ Info: Recompiling stale cache file /home/nicolai/.julia/compiled/v1.0/Distributions/xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1190
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for StatsFuns [4c63d2b9-4356-54db-8cca-17b64c39e42c]
│   exception = Required dependency SpecialFunctions [276daf66-3868-5448-9aa4-cd146d93841b] failed to load from a cache file.
└ @ Base loading.jl:969
┌ Info: Recompiling stale cache file /home/nicolai/.julia/compiled/v1.0/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1190


Plots.PyPlotBackend()

In [2]:
@everywhere src_path = "../Source/"
@everywhere include(src_path*"types.jl")
@everywhere include(src_path*"functions_msc.jl")
@everywhere include(src_path*"functions_neighbors.jl")
@everywhere include(src_path*"functions_types.jl")
@everywhere include(src_path*"functions_energy.jl")
@everywhere include(src_path*"functions_mc.jl")
@everywhere include(src_path*"functions_thermalization.jl")
@everywhere include(src_path*"functions_observables.jl")
@everywhere include(src_path*"functions_symmetries.jl")
include(src_path*"functions_plots_and_files.jl")

makeDirRec (generic function with 1 method)

In [3]:
THERM_FRAC = 1/10
DT_MAX = 10000
DT_MIN = 100
MEASURE_FILE = "measured.statelist"

@everywhere const two_pi = 2π

# We run a simulation with the parameters
g = 1.0    # Gauge coupling
ν = 0.3    # Anisotropy

# Other parameters
L = 24     # System length
L₃ = 24
N = L^2*L₃
T_list = [1.6, 1.7, 2.0]
T_list = reverse(T_list)
κ₅ = 1.0

# Calculate periodic boundary conditioned f s.t. fL ∈ N
f = 0.0/L
println("f set to $(f)")
sim = Controls(π-π/12, 1.0, 4.0)

M = 600
# Setup measurement storage
N_T = length(T_list)
#u⁺_avg_by_T = Array{Float64}(undef, N_T); u⁻_avg_by_T = Array{Float64}(undef, N_T)
#u⁺_err_by_T = Array{Float64}(undef, N_T); u⁻_err_by_T = Array{Float64}(undef, N_T)

#η⁺_avg_by_T = Array{Float64}(undef, N_T); η⁻_avg_by_T = Array{Float64}(undef, N_T)
#η⁺_err_by_T = Array{Float64}(undef, N_T); η⁻_err_by_T = Array{Float64}(undef, N_T)

#Υ_avg_by_T = Array{Float64}(undef, N_T)
#Υ_err_by_T = Array{Float64}(undef, N_T);

ρˣ₂_avg_by_T = Array{Float64}(undef, N_T); ρˣ₃_avg_by_T = Array{Float64}(undef, N_T); ρʸ₁_avg_by_T = Array{Float64}(undef, N_T)
ρˣ₂_err_by_T = Array{Float64}(undef, N_T); ρˣ₃_err_by_T = Array{Float64}(undef, N_T); ρʸ₁_err_by_T = Array{Float64}(undef, N_T)
ρʸ₃_avg_by_T = Array{Float64}(undef, N_T); ρᶻ₁_avg_by_T = Array{Float64}(undef, N_T); ρᶻ₂_avg_by_T = Array{Float64}(undef, N_T)
ρʸ₃_err_by_T = Array{Float64}(undef, N_T); ρᶻ₁_err_by_T = Array{Float64}(undef, N_T); ρᶻ₂_err_by_T = Array{Float64}(undef, N_T)

# Make ab inito un-correlated phases state
nw = max(1,nprocs()-1) # The number of low temperature states needed
init_syst_list = Array{SystConstants, 1}(undef, nw+1)
init_syst_list[1:nw] = [SystConstants(L, L₃, 1/g^2, ν, κ₅, f, 1/T_list[end]) for i = 1:nw] # Make nw low temp-states
init_syst_list[nw+1] = SystConstants(L, L₃, 1/g^2, ν, κ₅, f, 1/T_list[1])              # Make final high temp state.
init_ψ_list = [State(1, syst; u⁺=1.0, u⁻=0.0) for syst in init_syst_list]
init_sim_list = [copy(sim) for syst in init_syst_list];

f set to 0.0


In [4]:

# Then preform thermalization and measurement for each temperature individually in separate folders
@time for (i, T) in enumerate(T_list)
    println("\n\n*****************************************************************************\nEntering simulation of T = $(T)")
    flush(stdout)
    dir_name = "T_$(T)"
    mkcd(dir_name)

    # Start with fresh states
    ψ_ref = copy(init_ψ_list[end])
    ψ_w = [copy(ψ) for ψ in init_ψ_list[1:nw]]
    println("Thermalizing from high (T=$(1/ψ_ref.consts.β)) and low temperature (T=$(1/ψ_w[1].consts.β))")
    println([1/ψ_ref.consts.β, [1/ψ.consts.β for ψ in ψ_w]...])
    # Update to new temperature
    syst = SystConstants(L, L₃, 1/g^2, ν, κ₅, f, 1/T)
    ψ_ref.consts = syst
    for ψ in ψ_w
        ψ.consts = syst
    end

    println("Energies before thermalization:")
    println([E(ψ_ref)/N, [E(ψ)/N for ψ in ψ_w]...])
    # Thermalize states
    @time thermalized, t₀, ψ_ref, E_ref, sim_ref, ψ_w, E_w, sim_w = thermalizeLite!(ψ_ref, ψ_w, copy(sim); visible=true,
        STABILITY_CUTOFF=30000)
    # If we didn't manage to thermalize, try to go back to larger simulation constants
    #if !thermalized
    #    ψ_list = [ψ_ref, ψ_w...]
    #    sim_list = [copy(sim) for ψ in ψ_list]
    #    # Run for 1/4 of the previous thermalization time with larger constants
    #    ψ_list, E_matrix = nMCSEnergy(ψ_list, sim_list, floor(Int64, t₀/4)+1, [E(ψ) for ψ in ψ_list])

    println("Thermalized energies:")
    println([E(ψ_ref)/N, [E(ψ)/N for ψ in ψ_w]...])

    # Preform measurements by saving states to file.
    ψ_list = [ψ_ref, ψ_w...]
    Δt = max(DT_MIN, min(DT_MAX, ceil(Int64, t₀*THERM_FRAC)))
    println("Δt = $(Δt), which means that we will do in total $(M*Δt) MCS")
    
    (ρˣ₂_avg, ρˣ₂_err, ρˣ₃_avg, ρˣ₃_err, ρʸ₃_avg, ρʸ₃_err, ρʸ₁_avg, ρʸ₁_err,
    ρᶻ₁_avg, ρᶻ₁_err, ρᶻ₂_avg, ρᶻ₂_err, AVu⁺, AVu⁻, AVA, u⁺, AVE, Υx, Υy, Υz) = @time parallelMeasureGS!(ψ_list,
        sim_ref, M, Δt)

    ρˣ₂_avg_r, ρˣ₂_err_r = scientificRounding(ρˣ₂_avg, ρˣ₂_err);
    ρˣ₃_avg_r, ρˣ₃_err_r = scientificRounding(ρˣ₃_avg, ρˣ₃_err);
    ρʸ₁_avg_r, ρʸ₁_err_r = scientificRounding(ρʸ₁_avg, ρʸ₁_err);
    ρʸ₃_avg_r, ρʸ₃_err_r = scientificRounding(ρʸ₃_avg, ρʸ₃_err);
    ρᶻ₁_avg_r, ρᶻ₁_err_r = scientificRounding(ρᶻ₁_avg, ρᶻ₁_err);
    ρᶻ₂_avg_r, ρᶻ₂_err_r = scientificRounding(ρᶻ₂_avg, ρᶻ₂_err);

    println("ρˣ(k₂) = $(ρˣ₂_avg_r) ± $(ρˣ₂_err_r)")
    println("ρˣ(k₃) = $(ρˣ₃_avg_r) ± $(ρˣ₃_err_r)")
    println("ρʸ(k₁) = $(ρʸ₁_avg_r) ± $(ρʸ₁_err_r)")
    println("ρʸ(k₃) = $(ρʸ₃_avg_r) ± $(ρʸ₃_err_r)")
    println("ρᶻ(k₁) = $(ρᶻ₁_avg_r) ± $(ρᶻ₁_err_r)")
    println("ρᶻ(k₂) = $(ρᶻ₂_avg_r) ± $(ρᶻ₂_err_r)")

    # Save results to arrays
    global ρˣ₂_avg_by_T[i] = ρˣ₂_avg; global ρˣ₃_avg_by_T[i] = ρˣ₃_avg; global ρʸ₁_avg_by_T[i] = ρʸ₁_avg
    global ρˣ₂_err_by_T[i] = ρˣ₂_err; global ρˣ₃_err_by_T[i] = ρˣ₃_err; global ρʸ₁_err_by_T[i] = ρʸ₁_err
    global ρʸ₃_avg_by_T[i] = ρʸ₃_avg; global ρᶻ₁_avg_by_T[i] = ρᶻ₁_avg; global ρᶻ₂_avg_by_T[i] = ρᶻ₂_avg
    global ρʸ₃_err_by_T[i] = ρʸ₃_err; global ρᶻ₁_err_by_T[i] = ρᶻ₁_err; global ρᶻ₂_err_by_T[i] = ρᶻ₂_err


    # Use states to produce lists of order-parameter amplitudes
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    #u⁺_list, u⁻_list = measureMeanAmplitudes(ψ_measured)
    # Get averages and errors
    #u⁺_avg, u⁺_err = avgErr(u⁺_list); u⁻_avg, u⁻_err = avgErr(u⁻_list)
    # Get correlation times
    #u⁺_τ = M/effective_sample_size(u⁺_list); u⁻_τ = M/effective_sample_size(u⁻_list)
    #u⁺_avg_r, u⁺_err_r = scientificRounding(u⁺_avg, u⁺_err); u⁻_avg_r, u⁻_err_r = scientificRounding(u⁻_avg, u⁻_err)
    #println("⟨|η⁺|⟩ = $(u⁺_avg_r) ± $(u⁺_err_r)\t\tτ = $(u⁺_τ)")
    #println("⟨|η⁻|⟩ = $(u⁻_avg_r) ± $(u⁻_err_r)\t\tτ = $(u⁻_τ)")

    # Save result to array
    #u⁺_avg_by_T[i] = u⁺_avg; u⁻_avg_by_T[i] = u⁻_avg;
    #u⁺_err_by_T[i] = u⁺_err; u⁻_err_by_T[i] = u⁻_err;


    @everywhere GC.gc()

    cd("../")
end



*****************************************************************************
Entering simulation of T = 2.0
Thermalizing from high (T=2.0) and low temperature (T=1.6)
[2.0, 1.6, 1.6, 1.6]
Energies before thermalization:
[-3.0, -3.0, -3.0, -3.0]
Worker 1 initially thermalised after 2 steps
Worker 3 initially thermalised after 4 steps
Worker 1 initially thermalised after 109 steps
Worker 2 initially thermalised after 119 steps
Worker 3 initially thermalised after 102 steps
All workers initially thermalized after 119 steps

Controls after initial adjustment:
State	θmax			umax	Amax
1	2.8797932657906435	1.0	0.7901234567901234
2	2.8797932657906435	1.0	0.7901234567901234
3	2.8797932657906435	1.0	0.7901234567901234
4	2.8797932657906435	1.0	0.7901234567901234
With lowest AR: 0.32437427662037044,	highest AR: 0.3270670572916667

All workers flat.
  ⟨ΔE⟩{1} =	-109.52167566600951 ± 194.53491136179326
  ⟨ΔE⟩{2} =	-107.99046178853666 ± 148.0760624085391
  ⟨ΔE⟩{3} =	-157.05197137536902 ± 135.833025